In [1]:
import numpy as np 
import pandas as pd
import pickle
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers, Model,Input
from tensorflow.keras.layers import *
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,f1_score,classification_report

from prediction import predict 
from model import sentiment_model

In [2]:
train_path = 'https://raw.githubusercontent.com/malinphy/datasets/main/tweet_sentiment_extraction/twitter_sentiment_analysis/twitter_training.csv'
test_path = 'https://raw.githubusercontent.com/malinphy/datasets/main/tweet_sentiment_extraction/twitter_sentiment_analysis/twitter_validation.csv'

In [3]:
train_df = pd.read_csv(train_path,header = None).dropna().reset_index(drop= True)
test_df = pd.read_csv(test_path,header = None).reset_index(drop= True)
train_df = train_df.rename(columns={0: 'tweet_id', 1: 'entity',2:'sentiment',3:'content'})
test_df = test_df.rename(columns={0: 'tweet_id', 1: 'entity',2:'sentiment',3:'content'})

In [4]:
vocab_size = 45000
embed_dim = 32
input_len = 170

In [5]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=170
    )

vectorize_layer.adapt(train_df['content'])
train_tokens = vectorize_layer(train_df['content'])
test_tokens = vectorize_layer(test_df['content'])

In [6]:
input_len = tf.shape(train_tokens)[1]
corpus_size = len(vectorize_layer.get_vocabulary())

In [7]:
input_len

<tf.Tensor: shape=(), dtype=int32, numpy=170>

In [8]:
LE = LabelEncoder()
train_encoded_labels = LE.fit_transform(train_df['sentiment'])
test_encoded_labels = LE.transform(test_df['sentiment'])

In [9]:
def sentiment_model():
    input_len = 170
    vocab_size = 45000
    embed_dim = 32 
    input_layer = Input(shape=(input_len,), name = 'input_layer')
    emb_layer = Embedding(vocab_size, embed_dim, name = 'embedding_layer')(input_layer)
    flat_layer = Flatten(name = 'Flatten_layer')(emb_layer)
    d1_layer = Dense(128,activation = 'relu',name = 'd1_layer')(flat_layer)
    d2_layer = Dense(64,activation = 'relu',name = 'd2_layer')(d1_layer)
    d3_layer = Dense(32,activation = 'relu',name = 'd3_layer')(d2_layer)
    final_layer = Dense(4,activation = 'softmax',name = 'final_layer')(d3_layer)

    return Model(inputs = input_layer, outputs = final_layer)

model = sentiment_model()

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
    )

# history = model.fit(
#     train_tokens,
#     train_encoded_labels,
#     epochs = 10,
#     # validation_split = 0.2
#     )

# model.save_weights('sentiment_model.h5')

In [11]:
model.load_weights('sentiment_model.h5')

In [12]:
preds = tf.math.top_k(model.predict(test_tokens), k=1 )[1]

In [13]:
confusion_matrix(test_encoded_labels, preds)

array([[168,   2,   1,   1],
       [  2, 263,   0,   1],
       [  3,   3, 275,   4],
       [  1,   4,   3, 269]])

In [38]:
print('F1 SCORE',f1_score(test_encoded_labels, preds, average= 'micro'))
print(classification_report(test_encoded_labels, preds,  target_names = list(LE.inverse_transform([0,1,2,3]))  ))

F1 SCORE 0.975
              precision    recall  f1-score   support

  Irrelevant       0.97      0.98      0.97       172
    Negative       0.97      0.99      0.98       266
     Neutral       0.99      0.96      0.98       285
    Positive       0.98      0.97      0.97       277

    accuracy                           0.97      1000
   macro avg       0.97      0.98      0.97      1000
weighted avg       0.98      0.97      0.97      1000



In [16]:
pickle.dump({'config': vectorize_layer.get_config(),
             'weights': vectorize_layer.get_weights()}
            , open("tv_layer.pkl", "wb"))


from_disk = pickle.load(open("tv_layer.pkl", "rb"))
new_v = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
new_v.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
new_v.set_weights(from_disk['weights'])

output = open('LE.pkl', 'wb')
pickle.dump(LE, output)
output.close()

pkl_file = open('LE.pkl', 'rb')
le_departure = pickle.load(pkl_file) 
pkl_file.close()


In [17]:
new_test = (new_v('I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣'))

In [18]:
def predict(x):
    from_disk = pickle.load(open("tv_layer.pkl", "rb"))
    new_v = TextVectorization.from_config(from_disk['config'])
    # You have to call `adapt` with some dummy data (BUG in Keras)
    new_v.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
    new_v.set_weights(from_disk['weights'])
    
    pkl_file = open('LE.pkl', 'rb')
    le_departure = pickle.load(pkl_file) 
    pkl_file.close()

    # y = new_v(x)
    model.load_weights('sentiment_model.h5')
    test_sent = new_v(x)
    test_sent = tf.reshape(test_sent, shape = (1, input_len))
    y = np.argmax(model.predict(test_sent))

    return (le_departure.inverse_transform([y]))[0]

In [19]:
predict('I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣')

'Irrelevant'